<div>
<img src="https://www.nebrija.com/images/logos/logotipo-universidad-nebrija.jpg" width="200">
</div>

**MODELOS DE PROGRAMACION: MODELO ADIABATICO** -
Prof: Carmen Pellicer Lostao

# Constrained Scheduling

## Intro

This notebook solves a **binary constraint satisfaction problem (CSP)**. CSPs require that all a problem’s variables be assigned values that result in the satisfying of all constraints. Here, the constraints are a company’s policy for scheduling meetings:

- Constraint 1: During business hours, all meetings must be attended in person at the office.

- Constraint 2: During business hours, participation in meetings is mandatory.

- Constraint 3: Outside business hours, meetings must be teleconferenced.

- Constraint 4: Outside business hours, meetings must not exceed 30 minutes.

Solving such a CSP means **finding arrangements of meetings that meet all the constraints**.

The purpose of this example is to help a new user to formulate a constraint satisfaction problem using Ocean tools and solve it on a D-Wave quantum computer.

This example follows the [Workflow steps: Formulation and Sampling](https://docs.ocean.dwavesys.com/en/stable/overview/solving_problems.html#solving-problems) that describes the problem-solving workflow as consisting of two main steps:

1.  **Formulate the problem** as an objective function in a supported form of quadratic model (QM) and
2.   **Solve your QM** with a D-Wave solver.

In CSP problems we represent the **problem’s constraints as penalties** (small Binary Quadratic Modelss that have higher values for variable assignments that violate constraints) and **creates an objective function by summing all four penalty models**. Solvers that seek low-energy states are thus less likely to return meeting arrangements that violate constraints.

## Formulate the problem

D-Wave quantum computers solve binary quadratic models, so the first step is to express the problem with binary variables

### The Variables

In this scheduling example we use the following valued binary variables:

Problem Variables

|Variable | Represents | Value: 1 | Value: 0 |
| :---: | :---: | :---: | :---: |
|t |Time of day|Business hours|Non-business hours|
|v|Venue|Office|Teleconference|
|l|Length|Short (< 30 min)|Long|
|p|Participation|Mandatory|Optional|



<u>**NOTE**</u>: A slightly more complex problem might require variables with multiple values; for example, l could have values {30, 60, 120} representing the duration in minutes of meetings of several lengths. For such problems a discrete quadratic model (DQM) could be a better choice.

In general, problems with constraints are more simply solved using a constrained quadratic model (CQM).

For large numbers of variables and constraints, such problems can be hard. This example has four binary variables, so only $2^4=16$ possible meeting arrangements. As shown in the table below, it is a simple matter to work out all the combinations by hand to find solutions that meet all the constraints.

All Possible Meeting Options.

|Time of Day|Venue|Duration|Participation|Valid?|
| :---: | :---: | :---: | :---: | :---: |
|Business hours|Office|Short|Mandatory|Yes|
|Business hours|Office|Short|Optional|No (violates 2)|
|Business hours|Office|Long|Mandatory|Yes|
|Business hours|Office|Long|Optional|No (violates 2)|
|Business hours|Teleconference|Short|Mandatory|No (violates 1)|
|Business hours|Teleconference|Short|Optional|No (violates 1, 2)|
|Business hours|Teleconference|Long|Mandatory|No (violates 1)|
|Business hours|Teleconference|Long|Optional|No (violates 1, 2)|
|Non-business hours|Office|Short|Mandatory|No (violates 3)|
|Non-business hours|Office|Short|Optional|No (violates 3)|
|Non-business hours|Office|Long|Mandatory|No (violates 3, 4)|
|Non-business hours|Office|Long|Optional|No (violates 3, 4)|
|Non-business hours|Teleconference|Short|Mandatory|Yes|
|Non-business hours|Teleconference|Short|Optional|Yes|
|Non-business hours|Teleconference|Long|Mandatory|No (violates 4)|
|Non-business hours|Teleconference|Long|Optional|No (violates 4)|

### Represent Constraints as Penalties

You can represent constraints as BQMs using Penalty Models in many different ways.

- Constraint 1: During business hours, all meetings must be attended in person at the office.

This constraint requires that if $t=1$ (time of day is within business hours) then $v=1$ (venue is the office). A simple penalty function,
, is shown in the truth table below:

|$t$|$v$| $t-tv$|
| :---: | :---: | :---: |
|0|0|0|
|0|1|0|
|1|0|1|
|1|1|0|

<u>**NOTE**</u>: One way to derive such a penalty function is to start with the simple case of a Boolean operator: the AND constraint, $ab$, penalizes variable values $a=b=1$. To penalize $a=1,b=0$ , you need the penalty function $a¬b$
. For binary valued variables, you can substitute $~b=(1-b)$ into the penalty and get the penalty $a(1-b)$

- Constraint 2: During business hours, participation in meetings is mandatory.

This constraint requires that if $t=1$
 (time of day is within business hours) then $p=1$ (participation is mandatory). A penalty function is $t-tp$, analogous to constraint 1.

- Constraint 3: Outside business hours, meetings must be teleconferenced.

This constraint requires that if $t=0$ (time of day is outside business hours) then $v=0$ (venue is teleconference, not the office). A penalty function is $v-tv$, a reversal of constraint 1.

- Constraint 4: Outside business hours, meetings must not exceed 30 minutes.

This constraint requires that if $t=0$ (time of day is outside business hours) then $l=1$ (meeting length is short). A simple penalty function is $1+tl-t-l$
, as shown in the truth table below:

|$t$|$v$| $1+tl-t-l$    |
| :---: | :---: | :---: |
|0|0|1                  |
|0|1|0                  |
|1|0|0                  |
|1|1|0                  |

Penalty function $1+tl-t-l$ is
 sets a penalty of 1 for the case $t=l=0$
, representing a lengthy meeting outside business hours, which violates constraint 4. When incorporated in an objective function, solutions that violate constraint 4 do not yield minimal values

<u>**NOTE**</u>: One way to derive such a penalty function is to start with the simple case of a Boolean operator: the AND constraint, $ab$, penalizes variable values $a=b=1$. To penalize $a=0,b=0$ , you need the penalty function $~a~b$
. For binary valued variables, you can substitute $¬a=(1-a)$ and $¬b=(1-b)$ into the
penalty and get the penalty $(1-a)(1-b)=1-a-b+ab$

### Create a BQM

The total penalty for all four constraints is

$t-tv+t-tp+v-tv+1+tl-t-l = -2tv-tp+tl+t+v-l+1$

Ocean’s dimod enables the creation of BQMs. Below, the first list of terms are the linear terms and the second are the quadratic terms; the offset is set to 1; and the variable type is set to use
–valued binary variables

In [3]:
from dimod import BinaryQuadraticModel
bqm = BinaryQuadraticModel({'t': 1, 'v': 1, 'l': -1},       #linear
                           {'tv': -2, 'tl': 1, 'tp': -1},   #quadratic
                           1,                               #offset
                           'BINARY')                        #QUBO

## Solve the problem in a QPU

For small numbers of variables, even your computer’s CPU can solve CSPs quickly. Here you solve both classically on your CPU and on the quantum computer.

### Solving classically on a CPU

Before using a D-Wave quantum computer, it can sometimes be helpful to test code locally. Here, select one of Ocean software’s [classical samplers](https://docs.ocean.dwavesys.com/en/stable/docs_dimod/reference/sampler_composites/samplers.html) or [dimod sampler utilities](https://docs.ocean.dwavesys.com/en/stable/docs_dimod/reference/sampler_composites/index.html) to solve classically on a CPU.

Ocean’s `dimod` [ExactSolver()](https://docs.ocean.dwavesys.com/en/stable/docs_dimod/reference/sampler_composites/samplers.html#exact-solver) provides a sampler that simply returns the BQM’s value (energy) for every possible assignment of variable values.

Ocean's `dwave-samplers` [SimulatedAnnealing()](https://docs.ocean.dwavesys.com/en/stable/docs_samplers/index.html#simulated-annealing) provides a sampler that simulates a QPU

Valid solutions—assignments of variables that do not violate constraints—have the lowest value of the BQM (values of zero in the energy field below):


In [4]:
from dimod.reference.samplers import ExactSolver
sampler = ExactSolver()
sampleset_exact = sampler.sample(bqm)
print(sampleset_exact)
print(print(sampleset_exact.lowest(atol=.5)))

    l  p  t  v energy num_oc.
7   1  0  0  0    0.0       1
8   1  1  0  0    0.0       1
10  1  1  1  1    0.0       1
13  0  1  1  1    0.0       1
0   0  0  0  0    1.0       1
2   0  0  1  1    1.0       1
4   1  0  0  1    1.0       1
5   1  0  1  1    1.0       1
9   1  1  1  0    1.0       1
11  1  1  0  1    1.0       1
14  0  1  1  0    1.0       1
15  0  1  0  0    1.0       1
1   0  0  1  0    2.0       1
3   0  0  0  1    2.0       1
6   1  0  1  0    2.0       1
12  0  1  0  1    2.0       1
['BINARY', 16 rows, 16 samples, 4 variables]
   l  p  t  v energy num_oc.
0  1  0  0  0    0.0       1
1  1  1  0  0    0.0       1
2  1  1  1  1    0.0       1
3  0  1  1  1    0.0       1
['BINARY', 4 rows, 4 samples, 4 variables]
None


In [11]:
from dwave.samplers import SimulatedAnnealingSampler
sampler = SimulatedAnnealingSampler()
sampleset_sim = sampler.sample(bqm)
print(sampleset_sim)
print(print(sampleset_sim.lowest(atol=.5)))

   l  p  t  v energy num_oc.
0  0  1  1  1    0.0       1
['BINARY', 1 rows, 1 samples, 4 variables]
   l  p  t  v energy num_oc.
0  0  1  1  1    0.0       1
['BINARY', 1 rows, 1 samples, 4 variables]
None


The code below prints all those solutions (assignments of variables) for which the BQM has its minimum value.

In [12]:
for sample, energy in sampleset_exact.data(['sample', 'energy']):
    if energy==0:
        time = 'business hours' if sample['t'] else 'evenings'
        venue = 'office' if sample['v'] else 'home'
        length = 'short' if sample['l'] else 'long'
        participation = 'mandatory' if sample['p'] else 'optional'
        print("During {} at {}, you can schedule a {} meeting that is {}".format(time, venue, length, participation))

During evenings at home, you can schedule a short meeting that is optional
During evenings at home, you can schedule a short meeting that is mandatory
During business hours at office, you can schedule a short meeting that is mandatory
During business hours at office, you can schedule a long meeting that is mandatory


### Solving on a D-Wave Quantum Computer (QPU)

Now solve on a D-Wave system using sampler [DWaveSampler()](https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/samplers.html#dwavesampler) from Ocean software’s dwave-system. Also use its [EmbeddingComposite()](https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/composites.html#embeddingcomposite) composite to map your unstructured problem (variables such as t etc.) to the sampler’s graph structure (the QPU’s numerically indexed qubits) in a process known as minor-embedding. The next code sets up a D-Wave quantum computer as the sampler.

In [13]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler = EmbeddingComposite(DWaveSampler())

Because the sampled solution is probabilistic, returned solutions may differ between runs. Typically, when submitting a problem to a quantum computer, you ask for many samples, not just one. This way, you see multiple “best” answers and reduce the probability of settling on a suboptimal answer. Below, ask for 5000 samples.

In [14]:
sampleset = sampler.sample(bqm, num_reads=5000, label='SDK Examples - Scheduling')

The code below prints all those solutions (assignments of variables) for which the BQM has its minimum value and the number of times it was found.

In [15]:
print(sampleset.lowest(atol=.5))

   l  p  t  v energy num_oc. chain_.
0  1  1  0  0    0.0    1173     0.0
1  1  1  1  1    0.0     698     0.0
2  0  1  1  1    0.0    1312     0.0
3  1  0  0  0    0.0    1817     0.0
['BINARY', 4 rows, 5000 samples, 4 variables]


In [19]:
for sample, energy in sampleset.data(['sample', 'energy']):
    if energy==0:
        time = 'business hours' if sample['t'] else 'evenings'
        venue = 'office' if sample['v'] else 'home'
        length = 'short' if sample['l'] else 'long'
        participation = 'mandatory' if sample['p'] else 'optional'
        print("During {} at {}, you can schedule a {} meeting that is {}".format(time, venue, length, participation))

During evenings at home, you can schedule a short meeting that is optional
During business hours at office, you can schedule a short meeting that is mandatory
During business hours at office, you can schedule a long meeting that is mandatory
During evenings at home, you can schedule a short meeting that is mandatory


### EXERCICE

Execute this BQM in QPU Advantage2_prototype and inspect the embedding in the inspector tool

In [20]:
# Configuramos el sampler que nos interesa
miSampler = EmbeddingComposite(DWaveSampler(solver=dict(topology__type='zephyr')))

# Lo ejecutamos y mostramos los resultados
miSampleset = miSampler.sample(bqm, num_reads=5000, label='SDK Examples - Scheduling - Advantage2_prototype')
print(sampleset.lowest(atol=.5))

   l  p  t  v energy num_oc. chain_.
0  1  0  0  0    0.0    1008     0.0
1  1  1  1  1    0.0    1042     0.0
2  0  1  1  1    0.0    2039     0.0
3  1  1  0  0    0.0     911     0.0
['BINARY', 4 rows, 5000 samples, 4 variables]


In [21]:
# Imprimimos las frases que nos informan de estas soluciones
for sample, energy in miSampleset.data(['sample', 'energy']):
    if energy==0:
        time = 'business hours' if sample['t'] else 'evenings'
        venue = 'office' if sample['v'] else 'home'
        length = 'short' if sample['l'] else 'long'
        participation = 'mandatory' if sample['p'] else 'optional'
        print("During {} at {}, you can schedule a {} meeting that is {}".format(time, venue, length, participation))

During evenings at home, you can schedule a short meeting that is optional
During business hours at office, you can schedule a long meeting that is mandatory
During business hours at office, you can schedule a short meeting that is mandatory
During evenings at home, you can schedule a short meeting that is mandatory


Veamos lo que nos dice el inspector.

In [22]:
import dwave.inspector

dwave.inspector.show(miSampleset)

Serving Inspector on http://127.0.0.1:18001/?problemId=7a53345b-1cd3-4cd6-8d38-b23f7f85042b

'http://127.0.0.1:18001/?problemId=7a53345b-1cd3-4cd6-8d38-b23f7f85042b'

### Solving with DWave Hybrid Solvers

For big problems with many variables it is useful to use Hybrid solvers. The use of this solvers is quite straight forward and similiar with the SAPI interface.

Let's solve this problem with the [LeapHybridSampler](https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/samplers.html#leaphybridsampler), a class for using Leap’s cloud-based hybrid BQM solvers. Leap’s quantum-classical hybrid BQM solvers are intended to solve arbitrary application problems formulated as binary quadratic models (BQM).

In [23]:
from dwave.system import LeapHybridSampler

# Find a good solution
sampler = LeapHybridSampler()       
sampleset = sampler.sample(bqm) 

In [24]:
sampler.properties

{'minimum_time_limit': [[1, 3.0],
  [1024, 3.0],
  [4096, 10.0],
  [10000, 40.0],
  [30000, 200.0],
  [100000, 600.0],
  [1000000, 600.0]],
 'maximum_time_limit_hrs': 24.0,
 'maximum_number_of_variables': 1000000,
 'maximum_number_of_biases': 200000000,
 'parameters': {'time_limit': 'Maximum requested runtime in seconds.'},
 'supported_problem_types': ['bqm'],
 'category': 'hybrid',
 'version': '2.2',
 'quota_conversion_rate': 20}

In [25]:
print(sampleset.lowest(atol=.5))

   l  p  t  v energy num_oc.
0  1  0  0  0    0.0       1
['BINARY', 1 rows, 1 samples, 4 variables]


In [26]:
for sample, energy in sampleset.data(['sample', 'energy']):
    if energy==0:
        time = 'business hours' if sample['t'] else 'evenings'
        venue = 'office' if sample['v'] else 'home'
        length = 'short' if sample['l'] else 'long'
        participation = 'mandatory' if sample['p'] else 'optional'
        print("During {} at {}, you can schedule a {} meeting that is {}".format(time, venue, length, participation))

During evenings at home, you can schedule a short meeting that is optional


### EXERCICE

Include a new variable in this problem with new conditions that have to be satisfied. Formulate the problem and solve it

Vamos a suponer que estas reuniones pueden clasificarse en reuniones de directivos (aquellas que cuentan con un grado mayor de importancia), o rutinarias. La variable es binaria y la incluimos con las anteriores en la siguiente tabla:

|Variable | Representa | Valor: 1 | Valor: 0 |
| :---: | :---: | :---: | :---: |
|t |Momento del día|Horario de oficina|Fuera del horario de oficina|
|v|Modalidad|Presencial|Remota|
|l|Duración|Corta (< 30 min)|Larga|
|p|Participación|Obligatoria|Opcional|
|d|Tipo de reunión|De directivos|Rutinaria|

Con estas variables, debemos añadir unas normas para cuando la reunión sea de directivos (no hay modificaciones para el caso rutinario). Así,

- Las reuniones de directivos se realizarán siempre en horario de oficina.
- Las reuniones de directivos durarán más de 30 minutos.

Consecuentemente, si la reunión es de directivos las posibilidades son:

|Momento del día|Modalidad|Duración|Participación|¿Válida?|
| :---: | :---: | :---: | :---: | :---: |
|Horario de oficina|Presencial|Corta|Obligatoria|No (Falla 6)|
|Horario de oficina|Presencial|Corta|Opcional|No (Falla 2)|
|Horario de oficina|Presencial|Larga|Obligatoria|Sí|
|Horario de oficina|Presencial|Larga|Opcional|No (Falla 2)|
|Horario de oficina|Remota|Corta|Obligatoria|No (Fallan 1 y 6)|
|Horario de oficina|Remota|Corta|Opcional|No (Fallan 1, 2 y 6)|
|Horario de oficina|Remota|Larga|Obligatoria|No (Falla 1)|
|Horario de oficina|Remota|Larga|Opcional|No (Fallan 1, 2)|
|Fuera del horario de oficina|Presencial|Corta|Obligatoria|No (Fallan 5 y 6)|
|Fuera del horario de oficina|Presencial|Corta|Opcional|No (Fallan 5 y 6)|
|Fuera del horario de oficina|Presencial|Larga|Obligatoria|No (Falla 5)|
|Fuera del horario de oficina|Presencial|Larga|Opcional|No (Falla 5)|
|Fuera del horario de oficina|Remota|Corta|Obligatoria|No (Fallan 5 y 6))|
|Fuera del horario de oficina|Remota|Corta|Opcional|No (Fallan 5 y 6)|
|Fuera del horario de oficina|Remota|Larga|Obligatoria|No (Falla 5)|
|Fuera del horario de oficina|Remota|Larga|Opcional|No (Falla 5)|

Formularemos las nuevas restricciones.

#### Restricciones

- Restricción 5: Las reuniones de directivos se realizarán siempre en horario de oficina.

Esto quiere decir que si $d = 1$, entonces necesariamente $t = 1$. Debe penalizarse el caso $d = 1$, $t = 0$. Esto se consigue con la función de penalización $d - dt$. Esto es análogo a lo que ocurre en la restricción 1.

- Restricción 6: Las reuniones de directivos serán de larga duración.

Debemos penalizar el caso $d = 1$, $l = 1$. Una función de penalización sería $dl$, pues estamos tratando con el operador AND.

Con todo, tenemos que nuestro BQM es:

$$-2tv-tp+tl+t+v-l+1 + d-dt + dl = dl-dt-2tv-tp+tl+t+v+d-l+1$$

Creemos y resolvamos este BQM.

In [27]:
# Creamos el BQM
from dimod import BinaryQuadraticModel
miBQM = BinaryQuadraticModel({'t': 1, 'v': 1, 'd': 1, 'l': -1},                #linear
                           {'dl': 1, 'dt': -1, 'tv': -2, 'tl': 1, 'tp': -1},   #quadratic
                           1,                                                  #offset
                           'BINARY')                                           #QUBO

In [28]:
# Lo resolvemos en la QPU
from dwave.system import EmbeddingComposite,DWaveSampler
samplerBQM = EmbeddingComposite(DWaveSampler())
samplesetBQM = samplerBQM.sample(miBQM, num_reads=5000, label='Ejemplo BQM ejercicio')

# Imprimiremos solo los resultados cuyo valor energético sea menor que 0.5
print(samplesetBQM.lowest(atol=.5))

   d  l  p  t  v energy num_oc. chain_.
0  0  1  0  0  0    0.0    1570     0.0
1  0  1  1  1  1    0.0     354     0.0
2  0  1  1  0  0    0.0     908     0.0
3  0  0  1  1  1    0.0     673     0.0
4  1  0  1  1  1    0.0    1495     0.0
['BINARY', 5 rows, 5000 samples, 5 variables]


Nos preguntamos si se cumple lo visto en las tablas. Para ello, imprimimos la interpretación de los resultados a continuación.

In [29]:
for muestra, energia in samplesetBQM.data(['sample', 'energy']):
    if energia==0:
        time = 'horario de oficina' if muestra['t'] else 'las tardes (fuera de horario de oficina)'
        venue = 'presencial' if muestra['v'] else 'remota'
        length = 'corta' if muestra['l'] else 'larga'
        participation = 'obligatoria' if muestra['p'] else 'opcional'
        type = 'de directivos' if muestra['d'] else 'rutinaria'
        print("En {}, puedes organizar una reunión {} {} de {} duración y de participación {}.".format(time, type, venue, length, participation))

En las tardes (fuera de horario de oficina), puedes organizar una reunión rutinaria remota de corta duración y de participación opcional.
En horario de oficina, puedes organizar una reunión rutinaria presencial de corta duración y de participación obligatoria.
En las tardes (fuera de horario de oficina), puedes organizar una reunión rutinaria remota de corta duración y de participación obligatoria.
En horario de oficina, puedes organizar una reunión rutinaria presencial de larga duración y de participación obligatoria.
En horario de oficina, puedes organizar una reunión de directivos presencial de larga duración y de participación obligatoria.


In [30]:
import dimod
import numpy as np
from dwave.system import LeapHybridCQMSampler
import numpy as np
import matplotlib
matplotlib.use("agg")
import matplotlib.pyplot as plt

In [67]:
N = 13*17
nx = np.ceil(np.log2(np.floor(np.sqrt(N)))) - 1
ny = np.ceil(np.log2(np.floor(N/3))) - 1

total_qubits = nx+ny

In [68]:
variables = [dimod.Integer("x"),dimod.Integer("y")]

In [69]:
variables

[QuadraticModel({'x': 1.0}, {}, 0.0, {'x': 'INTEGER'}, dtype='float64'),
 QuadraticModel({'y': 1.0}, {}, 0.0, {'y': 'INTEGER'}, dtype='float64')]

In [70]:
variables[0].set_upper_bound("x",2**nx)
variables[1].set_upper_bound("y",2**ny)

In [71]:
cqm = dimod.ConstrainedQuadraticModel()

In [72]:
cqm.set_objective(N - (2*variables[0]+1)*(2*variables[1]+1))

In [73]:
print(cqm)

Constrained quadratic model: 2 variables, 0 constraints, 3 biases

Objective
  220 - 2*Integer('x') - 2*Integer('y') - 4*Integer('x')*Integer('y')

Constraints

Bounds
  0.0 <= Integer('x') <= 8.0
  0.0 <= Integer('y') <= 64.0



In [74]:
cqm.add_constraint(N - (2*variables[0]+1)*(2*variables[1]+1)>=0)

'ce7fd68'

In [75]:
print(cqm)

Constrained quadratic model: 2 variables, 1 constraints, 6 biases

Objective
  220 - 2*Integer('x') - 2*Integer('y') - 4*Integer('x')*Integer('y')

Constraints
  ce7fd68: 220 - 2*Integer('x') - 2*Integer('y') - 4*Integer('x')*Integer('y') >= 0.0

Bounds
  0.0 <= Integer('x') <= 8.0
  0.0 <= Integer('y') <= 64.0



In [76]:
# Definimos el sampler
sampler = LeapHybridCQMSampler()

# Resolvemos el CQM
sampleset = sampler.sample_cqm(cqm)

# Tomamos las soluciones factibles
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)   

In [77]:
best = feasible_sampleset.first.sample
print([2*best[ind]+1 for ind in ["x","y"] ])

[13.0, 17.0]
